In [13]:
import torch
from transformers import Data2VecTextForSequenceClassification, AutoTokenizer
from PIL import Image
import requests

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [15]:
version = "facebook/data2vec-text-base"
text = ["Hello, my dog is cute", "Hello, my dog is ugly"]

# AutoTokenizer

In [16]:
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(version)
tokenizer

RobertaTokenizerFast(name_or_path='facebook/data2vec-text-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## special ids and tokens

In [17]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 2, 3, 1, 50264]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [18]:
tokenizer.mask_token_id

50264

## processor

In [19]:
inputs = tokenizer(
    text = text,                # 可以为列表或单个string
    return_tensors = "pt",      # 返回数据格式 np pt tf jax
    padding = True,             # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,  # 如果使用max_length要将padding设置为 "max_length"
    add_special_tokens = True,  # text添加特殊key
).to(device, torch.float16)
inputs

{'input_ids': tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2],
        [    0, 31414,     6,   127,  2335,    16, 11355,     2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [20]:
inputs["input_ids"]

tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2],
        [    0, 31414,     6,   127,  2335,    16, 11355,     2]],
       device='cuda:0')

# Data2VecTextForSequenceClassification

Data2VecText Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks.

In [21]:
model: Data2VecTextForSequenceClassification = Data2VecTextForSequenceClassification.from_pretrained(version, torch_dtype=torch.float16).to(device)
model

Some weights of Data2VecTextForSequenceClassification were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data2VecTextForSequenceClassification(
  (data2vec_text): Data2VecTextModel(
    (embeddings): Data2VecTextForTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Data2VecTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x Data2VecTextLayer(
          (attention): Data2VecTextAttention(
            (self): Data2VecTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Data2VecTextSelfOutput(
              (dense): Linear(in_fea

In [22]:
model.eval()
with torch.inference_mode():
    outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0461,  0.0457],
        [-0.0459,  0.0477]], device='cuda:0'), hidden_states=None, attentions=None)

In [23]:
logits = outputs.logits
print(logits.shape)
print(logits)

torch.Size([2, 2])
tensor([[-0.0461,  0.0457],
        [-0.0459,  0.0477]], device='cuda:0')


In [27]:
logits.argmax(dim=1)

tensor([1, 1], device='cuda:0')